# First stab at GBR coral cover

## conceptual workflow 

In the longer run the aim is to sample geo-median imagery and other covariate data (depth, slope, etc) around in situ sample data dates, and then predict time-series coral cover  

For the moment, to get the hang of it, use a once-off longer time period mosaic and do categorical maps + coral cover. Something like:  
    
    1. Generate a once off nice composite, maybe a 2-3 year period (2017-2019? - gives access to Sentinel-2 Level 2?)  
    2. Maybe segmentation - **optional to begin with**  
    3. Sample from the whole trianing data set (*reef cloud*) [but for the moment sample the same time period and AOI as the mosaic]  
    4. Fit a ML model (e.g. RF or SVM etc.)  
    5. Explore and validate  
    6. experiment with OBIA implementation for relationship/neighbourhood rules  

**Cummulaive Q's:**
- once settled on a larger area, should I export the temporal composites (and segmentation etc.)?
- what are the costs for reprojection? Can you set to use the native/nominal projection of data?
- RE the `query` dictionary passed to the loading function or loading helper: does the `output_crs` only apply to output after the processing subsquent to the load, or does it always apply to every time-step found in the collection?  
- what happens when you stop execution while dask chunks are processing/queued (seems not to like that...)?  
- for lazy eval, is it quicker to `.load()`/`.compute()` just the individual bands, several at one, or the whole lot?  And if several at once, how to do that via the slot method (i.e. `lazy_dat.blue.load()` --> `lazy_day.some_bands.load()`  

    

#### Front matter

In [ ]:
%matplotlib inline

import subprocess as sp
import sys
import datacube
import rasterio

import shapely

import pydotplus

import numpy as np
import geopandas as gpd

from odc.ui import with_ui_cbk
from odc.algo import to_f32, xr_geomedian, int_geomedian

from io import StringIO
from sklearn import tree
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from IPython.display import Image
from datacube.utils import geometry
from datacube.utils.cog import write_cog

sys.path.append("../../Scripts")
from dea_datahandling import load_ard
from dea_plotting import rgb
from dea_dask import create_local_dask_cluster
from dea_classificationtools import collect_training_data
from dea_classificationtools import predict_xr
from dea_plotting import map_shapefile

# start the cluster
create_local_dask_cluster()

import warnings
warnings.filterwarnings('ignore') ## why?

dc = datacube.Datacube(app="coral_cover")

## Load data

#### Image composite (temporal)

In [ ]:
query = {
    "x": (151.48837, 152.22467),
    "y": (-23.17189, -23.60526),
    "time": ("2019-06-01", "2019-06-05"),
    "group_by": "solar_day",
    "resolution": (-10, 10),
    'measurements': ['nbar_coastal_aerosol', 'nbar_blue', 'nbar_green', 'nbar_red', 'nbar_nir_1'],
    'output_crs': 'EPSG:3577'
}

s2_collection = load_ard(dc=dc,
                         products=["s2a_ard_granule", "s2b_ard_granule"],
                         dask_chunks={"time": 1, "x": 2000, "y": 2000},
                         dtype='native',
                         **query)

print(s2_collection)

s2_geomedian = int_geomedian(s2_collection)
# compute - ideally i'd like to be able to load the bands at this point - possible? e.g. s2_geomedian.some_bands.load()
s2_geomedian = s2_geomedian.load()

In [ ]:
print(s2_geomedian)
rgb(s2_geomedian, bands = ['nbar_blue', 'nbar_green', 'nbar_red'])

#### In situ data

In [ ]:
coral_dat = gpd.read_file("gbr_reefcloud_coral.shp")

# fix for the meantime to ensure projections line up
#coral_dat = coral_dat.to_crs('EPSG:3577')

In [ ]:
print(type(coral_dat))
print(coral_dat.head())
print(coral_dat.shape[0])
class_field = "class_num"

print(coral_dat.crs)

'''
# select required cols?
coral_dat_lite = gpd.GeoDataFrame(coral_dat[[class_field]])
coral_dat_lite['geometry'] = coral_dat.geometry
print(coral_dat_lite.head())
'''

In [ ]:
# just subsample and plot to check
coral_dat['random'] = np.random.uniform(0, 1, coral_dat.shape[0])
coral_dat_sample = coral_dat[coral_dat.random > 0.999]
coral_dat_sample = coral_dat_sample.reset_index(drop=True)

#coral_dat_buffs = coral_dat_sample
#coral_dat_buffs['geometry'] = coral_dat_sample.geometry.buffer(0.0001)

print(coral_dat_sample.shape)
#print(coral_dat_buffs.shape)

In [ ]:
map_shapefile(coral_dat_sample, attribute = class_field)
#map_shapefile(coral_dat_buffs, attribute = class_field)

### Sample trianing data

- Directly from imagery (rather than from the geomedian generated above), with future in mind

In [ ]:
samp_query = {
    "time": ("2019-06-01", "2019-06-30"),
    "resolution": (-10, 10),
    'group_by' :'solar_day',
    'measurements': ['nbar_coastal_aerosol', 'nbar_blue', 'nbar_green', 'nbar_red', 'nbar_nir_1']
}

column_names, model_input = collect_training_data(
                                    gdf=coral_dat_sample, # use the subset while developing
                                    products=["s2a_ard_granule", "s2b_ard_granule"],
                                    dc_query=samp_query,
                                    ncpus=1,
                                    custom_func=None,
                                    field=class_field,
                                    calc_indices=None,
                                    reduce_func='median',
                                    drop=False,
                                    zonal_stats='mean')


In [ ]:
print(column_names)
print(model_input[0:5])